In [14]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import os

# Load JSON data
with open('movie_summaries.json', 'r') as file:
    movie_data = json.load(file)

def get_movie_summary(movie_title):
    # Fetch the summary for the given movie title
    summary = movie_data.get(movie_title, "Sorry, no summary available for this movie.")
    return summary
# Get a list of movie titles for the dropdown
movie_titles = list(movie_data.keys())

summaries = [movie_data[title] for title in movie_titles]

# Vectorize summaries using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(summaries)

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Recommendation function
def recommend_movies(movie_title, num_recommendations=5):
    if movie_title not in movie_titles:
        return "Movie not found in the database."
    
    movie_idx = movie_titles.index(movie_title)
    sim_scores = list(enumerate(cosine_sim[movie_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]  # Exclude the movie itself
    
    recommended_titles = [movie_titles[i] for i, _ in sim_scores]
    recommendations = {title: movie_data[title] for title in recommended_titles}
    return recommendations

# Gradio interface
def display_recommendations(movie_title):
    recommendations = recommend_movies(movie_title)
    if isinstance(recommendations, str):
        return recommendations
    else:
        output = f"If you like, {movie_title}, you may also like: \n\n"
        for title, summary in recommendations.items():
            output += f"**{title}**\n\n"
        return output



# Create a Gradio interface
title_dropdown = gr.Dropdown(choices=movie_titles, label="Select a movie title")
summary_output = gr.Textbox(label="Movie Summary")

interface = gr.Interface(
    fn=display_recommendations,
    inputs=title_dropdown,
    outputs="markdown",
    title="Movie Recommendation System",
    theme="Glass",
    css= ".gradio-container {background: url('file=film_reel.png'), #f0f0f0; background-size: cover;}",
    description="Select a movie title to get a summary of reviews."
)

interface.launch(allowed_paths=[os.path.dirname('film_reel.png')])

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
